In [298]:
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px 
import glob
import numpy as np

In [299]:
NODE_COUNT=4
USE_META_LEARNING=True

In [300]:
if USE_META_LEARNING:
    results = [r for r in glob.glob(f"./../results/*AVERAGE_AGGREGATION_{NODE_COUNT}*/") if not "CLIPPED" in r]
    plot_file_term="MetaL_method"
else:
    results = glob.glob(f"./../results/NO_META_LEARNING*{NODE_COUNT}*/")
    plot_file_term="FL_method"
len(results)

7

In [301]:
results

['./../results/DIRECT_MEANS_AVERAGE_AGGREGATION_4_NODES/',
 './../results/GRADUAL_GENETIC_ALGORITHM_MATING_AVERAGE_AGGREGATION_4_NODES/',
 './../results/DIRECT_GENETIC_ALGORITHM_MATING_AVERAGE_AGGREGATION_4_NODES/',
 './../results/GRADUAL_SCORE_WEIGHTED_MEANS_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_AVERAGE_AGGREGATION_4_NODES/',
 './../results/GRADUAL_MEANS_AVERAGE_AGGREGATION_4_NODES/',
 './../results/DIRECT_SCORE_WEIGHTED_MEANS_AVERAGE_AGGREGATION_4_NODES/']

In [302]:
iter_dfs = [pickle.load(open(file +"/iter_df.pkl","rb")) for file in results]
training_dfs = [pickle.load(open(file +"/training_df.pkl","rb")) for file in results]

if not USE_META_LEARNING:
    aggregation_methods=["_".join(file.split("_")[3:-2]) for file in results]
else:
    aggregation_methods=["_".join(file.split("/")[-2].split("_")[:-4]) for file in results]

In [303]:
aggregation_methods

['DIRECT_MEANS',
 'GRADUAL_GENETIC_ALGORITHM_MATING',
 'DIRECT_GENETIC_ALGORITHM_MATING',
 'GRADUAL_SCORE_WEIGHTED_MEANS',
 'NO_META_LEARNING',
 'GRADUAL_MEANS',
 'DIRECT_SCORE_WEIGHTED_MEANS']

In [304]:
iter_dfs[0]

,curr_score_0,iter_counter_0,DG_rotation_0,timestamp_0,curr_score_1,iter_counter_1,DG_rotation_1,timestamp_1,curr_score_2,iter_counter_2,DG_rotation_2,timestamp_2,curr_score_3,iter_counter_3,DG_rotation_3,timestamp_3
0,NaN,1,1,2024-02-27 21:21:32.624212,NaN,1,1,2024-02-27 21:21:33.229515,NaN,1,0,2024-02-27 21:21:33.228963,NaN,1,1,2024-02-27 21:21:32.763848
1,0.784333,2,1,2024-02-27 21:21:42.076444,0.740667,2,2,2024-02-27 21:21:43.031675,0.781667,2,0,2024-02-27 21:21:40.065587,0.784333,2,1,2024-02-27 21:21:43.672847
2,0.784333,3,1,2024-02-27 21:21:42.698057,0.737333,3,2,2024-02-27 21:21:43.672147,0.780000,3,0,2024-02-27 21:21:40.693836,0.782333,3,1,2024-02-27 21:21:44.300626
3,0.798333,4,2,2024-02-27 21:21:43.320241,0.751667,4,3,2024-02-27 21:21:44.296862,0.786333,4,0,2024-02-27 21:21:41.331999,0.788000,4,1,2024-02-27 21:21:44.919726
4,0.802667,5,2,2024-02-27 21:21:43.944055,0.752667,5,3,2024-02-27 21:21:44.919443,0.787333,5,0,2024-02-27 21:21:41.957836,0.786000,5,1,2024-02-27 21:21:45.544190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.756667,996,280,2024-02-27 21:34:19.410459,0.759667,996,627,2024-02-27 21:37:49.539103,0.764000,996,197,2024-02-27 21:34:31.071311,0.774000,996,218,2024-02-27 21:34:34.997010
996,0.755000,997,280,2024-02-27 21:34:20.041734,0.757667,997,627,2024-02-27 21:37:50.165865,0.762333,997,197,2024-02-27 21:34:31.715640,0.774667,997,218,2024-02-27 21:34:35.630599
997,0.745000,998,281,2024-02-27 21:34:20.675533,0.761000,998,627,2024-02-27 21:37:50.806136,0.749333,998,198,2024-02-27 21:34:32.364139,0.771333,998,218,2024-02-27 21:34:36.263532
998,0.735667,999,282,2024-02-27 21:34:21.300149,0.761333,999,627,2024-02-27 21:37:51.422952,0.807333,999,198,2024-02-27 21:34:37.982021,0.771667,999,218,2024-02-27 21:34:36.890948


In [305]:
scores=[]
for aggregation_method,iter_df in zip(aggregation_methods,iter_dfs):
    rel_columns= [c for c in iter_df.columns if "curr_score_" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = iter_df[rel_columns]
    scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [306]:
score_df=pd.concat(scores,axis=1)
score_df

,DIRECT_MEANS_curr_score_0,DIRECT_MEANS_curr_score_1,DIRECT_MEANS_curr_score_2,DIRECT_MEANS_curr_score_3,GRADUAL_GENETIC_ALGORITHM_MATING_curr_score_0,GRADUAL_GENETIC_ALGORITHM_MATING_curr_score_1,GRADUAL_GENETIC_ALGORITHM_MATING_curr_score_2,GRADUAL_GENETIC_ALGORITHM_MATING_curr_score_3,DIRECT_GENETIC_ALGORITHM_MATING_curr_score_0,DIRECT_GENETIC_ALGORITHM_MATING_curr_score_1,...,NO_META_LEARNING_curr_score_2,NO_META_LEARNING_curr_score_3,GRADUAL_MEANS_curr_score_0,GRADUAL_MEANS_curr_score_1,GRADUAL_MEANS_curr_score_2,GRADUAL_MEANS_curr_score_3,DIRECT_SCORE_WEIGHTED_MEANS_curr_score_0,DIRECT_SCORE_WEIGHTED_MEANS_curr_score_1,DIRECT_SCORE_WEIGHTED_MEANS_curr_score_2,DIRECT_SCORE_WEIGHTED_MEANS_curr_score_3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.784333,0.740667,0.781667,0.784333,0.740667,0.784333,0.784333,0.781667,0.781667,0.784333,...,0.784333,0.784333,0.781667,0.784333,0.784333,0.740667,0.784333,0.784333,0.740667,0.781667
2,0.784333,0.737333,0.780000,0.782333,0.737333,0.784333,0.782333,0.780000,0.780000,0.782333,...,0.782333,0.784333,0.780000,0.782333,0.784333,0.737333,0.782333,0.784333,0.737333,0.780000
3,0.798333,0.751667,0.786333,0.788000,0.751667,0.798333,0.788000,0.786333,0.786333,0.788000,...,0.788000,0.798333,0.786333,0.788000,0.798333,0.751667,0.788000,0.798333,0.751667,0.786333
4,0.802667,0.752667,0.787333,0.786000,0.752667,0.802667,0.786000,0.787333,0.787333,0.786000,...,0.786000,0.802667,0.787333,0.786000,0.802667,0.752667,0.786000,0.802667,0.752667,0.787333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.756667,0.759667,0.764000,0.774000,0.500000,0.500000,0.500000,0.500000,0.733667,0.796333,...,0.789333,0.641000,0.748000,0.748000,0.811000,0.829000,0.776667,0.791000,0.752667,0.760667
996,0.755000,0.757667,0.762333,0.774667,0.500000,0.500000,0.500000,0.500000,0.731333,0.800333,...,0.781333,0.644667,0.810000,0.755000,0.811667,0.817667,0.775000,0.789333,0.749333,0.764000
997,0.745000,0.761000,0.749333,0.771333,0.500000,0.500000,0.500000,0.500000,0.707000,0.799000,...,0.786333,0.641667,0.814333,0.748667,0.807333,0.823000,0.778000,0.780667,0.748333,0.744000
998,0.735667,0.761333,0.807333,0.771667,0.500000,0.500000,0.500000,0.500000,0.795000,0.800333,...,0.785000,0.639000,0.811000,0.749667,0.797000,0.825000,0.777667,0.773333,0.750000,0.741667


In [307]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    rel_col=[c for c in score_df.columns if aggregation_method == c[:-13]]
    result_dict=dict()
    by_node_result_average_dict=dict()
    for col in rel_col:
        by_node_result_average_dict[col.split("_")[-1]]=score_df[col].mean()
    result_dict["by_node_results"]=score_df[rel_col].mean(axis=0)
    result_dict["by_node_results_std"]=score_df[rel_col].std(axis=0)
    result_dict["total_average"]=score_df[rel_col].mean().mean()
    result_dict["total_std"]=score_df[rel_col].mean().std()
    result_dict["average_in_time"]= score_df[rel_col].mean(axis=1)
    average_score_dict[aggregation_method]=result_dict

In [308]:
aggregation_names=[]
aggregation_averages=[]
aggregation_stds=[]
by_node_averages=[]
by_node_stds=[]
average_in_time=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    aggregation_averages.append(val["total_average"])
    aggregation_stds.append(val["total_std"])
    by_node_averages.append(val["by_node_results"].values)
    by_node_stds.append(val["by_node_results_std"])
    average_in_time.append(val["average_in_time"])
sorted_tuples=sorted(zip(aggregation_names,aggregation_averages,by_node_averages,average_in_time,aggregation_stds, by_node_stds),key=lambda x: x[1])
aggregation_names = [val[0] for val in sorted_tuples]
aggregation_averages = [val[1] for val in sorted_tuples]
by_node_averages = [val[2] for val in sorted_tuples]
average_in_time = [val[3] for val in sorted_tuples]
aggregation_stds = [val[4] for val in sorted_tuples]
by_node_stds = [val[5] for val in sorted_tuples]

In [309]:
n_nodes=len(by_node_averages[0])
n_nodes

4

In [310]:
by_node_averages

[array([0.6481001 , 0.65956523, 0.70220888, 0.70968368]),
 array([0.73338105, 0.71225893, 0.72534835, 0.71106006]),
 array([0.78060093, 0.71947147, 0.70900901, 0.76468569]),
 array([0.780668  , 0.6968632 , 0.7269039 , 0.78123524]),
 array([0.78258392, 0.76986887, 0.74892593, 0.7562716 ]),
 array([0.72984518, 0.76303971, 0.78367668, 0.78233534]),
 array([0.78278111, 0.78086787, 0.76298765, 0.76006707])]

In [311]:
processed_aggregation_names=[" ".join(name.split("_")[:-1]) for name in aggregation_names]
processed_aggregation_names

['GRADUAL GENETIC ALGORITHM',
 'NO META',
 'GRADUAL SCORE WEIGHTED',
 'DIRECT SCORE WEIGHTED',
 'DIRECT GENETIC ALGORITHM',
 'DIRECT',
 'GRADUAL']

In [312]:
aggregation_names

['GRADUAL_GENETIC_ALGORITHM_MATING',
 'NO_META_LEARNING',
 'GRADUAL_SCORE_WEIGHTED_MEANS',
 'DIRECT_SCORE_WEIGHTED_MEANS',
 'DIRECT_GENETIC_ALGORITHM_MATING',
 'DIRECT_MEANS',
 'GRADUAL_MEANS']

In [313]:
name_mapping={"FIXED_CLIPPED_AVERAGE_AGGREGATION": "Fixed clip",
 "MEDIAN_AGGREGATION": "Median",
 "DYNAMIC_CLIPPED_AVERAGE_AGGREGATION": "Dynamic clip",
 "WEIGHTED_AVERAGE_PUNISHING_UPDATES": "Punish updates",
 "AVERAGE_AGGREGATION": "Simple average",
 "WEIGHTED_AVERAGE_RECENCY": "Receny weighted",
 "WEIGHTED_AVERAGE_PERFORMANCE_SCORES": "Performance weighted",
 "NO_FEDERATED_LEARNING": "No FL",
 "NO_META_LEARNING": "No MetaL",
 "DIRECT_SCORE_WEIGHTED_MEANS":"Direct weighted",
 "DIRECT_GENETIC_ALGORITHM_MATING":"Direct sampled",
 "GRADUAL_SCORE_WEIGHTED_MEANS":"Gradual weighted",
 "GRADUAL_MEANS":"Gradual regular",
 "GRADUAL_GENETIC_ALGORITHM_MATING":"Gradual sampled",
 "DIRECT_MEANS":"Direct regular",}
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Gradual sampled',
 'No MetaL',
 'Gradual weighted',
 'Direct weighted',
 'Direct sampled',
 'Direct regular',
 'Gradual regular']

In [314]:
fig=go.Figure()
for node in range(n_nodes):
    fig.add_trace(go.Bar(x=processed_aggregation_names, y=[val[node] for val in by_node_averages],error_y=dict(type="data",array=[val[node] for val in by_node_stds]),name=f"Node {node}"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=aggregation_averages,error_y=dict(type="data",array=aggregation_stds),name="Average"))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Average_score_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

/tmp/ipykernel_31458/1946294216.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [315]:
fig=go.Figure()
for idx,name in enumerate(processed_aggregation_names):
    fig.add_trace(go.Scatter(y=average_in_time[idx][1:] ,name=name))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=600,legend=dict( orientation="h",y=1.2),margin=dict(t=150,r=10))
fig.update_xaxes(title="Data generation iteration")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Performance_in_time_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [316]:
training_dfs[0].head()

,nesterov_mean_0,nesterov_std_0,batch_size_mean_0,batch_size_std_0,learning_rate_mean_0,learning_rate_std_0,n_epochs_mean_0,n_epochs_std_0,momentum_mean_0,momentum_std_0,...,n_runs_3,n_epochs_3,weights_3,iter_count_3,pre_training_score_3,scores_3,best_score_3,best_hyper_params_3,best_model_weights_3,timestamp_3
0,0.65,0.3,72.70,14.398264,-1.653113,0.259503,7.80,1.630951,0.761223,0.152821,...,9.0,0.777778,"[[[-1.434711480140686, -2.503401792049408, -0....",553.0,0.631667,"[0.7966666666666666, 0.48333333333333334, 0.76...",0.820000,"{'learning_rate': 0.04161226517131532, 'batch_...","[[[-1.0698978900909424, -2.9982693195343018, -...",2024-02-27 21:28:46.327860
1,0.90,0.3,70.95,20.000000,-1.520145,0.303889,7.05,3.000000,0.641462,0.232673,...,7.0,1.142857,"[[[-0.11339520083533393, -1.1785642786158457, ...",66.0,0.650000,"[0.6716666666666666, 0.7933333333333333, 0.783...",0.801667,"{'learning_rate': 0.10756377417206836, 'batch_...","[[[0.0625004693865776, -2.463059186935425, -1....",2024-02-27 21:22:36.775181
2,0.90,0.3,65.65,13.439773,-1.049988,1.000000,6.75,3.000000,0.634704,0.185360,...,6.0,1.333333,"[[[-0.033907658979296686, -3.3272716403007507,...",871.0,0.526667,"[0.8, 0.4866666666666667, 0.4866666666666667, ...",0.800000,"{'learning_rate': 0.3174901793162733, 'batch_s...","[[[1.0329828262329102, -2.090285301208496, 0.2...",2024-02-27 21:32:58.641297
3,1.00,0.0,67.80,20.000000,-0.936866,0.482394,7.20,1.860108,0.608066,0.300000,...,7.0,1.000000,"[[[-0.09030527350577441, -1.3263967430049723, ...",104.0,0.653333,"[0.77, 0.7833333333333333, 0.7683333333333333,...",0.793333,"{'learning_rate': 0.01945325229846966, 'batch_...","[[[-0.034223176538944244, -1.5891544818878174,...",2024-02-27 21:23:05.444788
4,1.00,0.0,61.05,19.586922,-1.069607,1.000000,8.30,3.000000,0.656430,0.148672,...,6.0,1.000000,"[[[-0.5917221996933222, -2.681718701124191, -0...",760.0,0.506667,"[0.775, 0.7633333333333333, 0.6233333333333333...",0.803333,"{'learning_rate': 0.07978795825570169, 'batch_...","[[[0.481827974319458, -2.6253433227539062, -0....",2024-02-27 21:31:32.735383


In [317]:
trainig_scores=[]
for aggregation_method,training_df in zip(aggregation_methods,training_dfs):
    rel_columns= [c for c in training_df.columns if "score" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = training_df[rel_columns]
    trainig_scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [318]:
training_scores_df=pd.concat(trainig_scores,axis=1)
training_scores_df

,DIRECT_MEANS_pre_training_score_0,DIRECT_MEANS_scores_0,DIRECT_MEANS_best_score_0,DIRECT_MEANS_pre_training_score_1,DIRECT_MEANS_scores_1,DIRECT_MEANS_best_score_1,DIRECT_MEANS_pre_training_score_2,DIRECT_MEANS_scores_2,DIRECT_MEANS_best_score_2,DIRECT_MEANS_pre_training_score_3,...,DIRECT_SCORE_WEIGHTED_MEANS_best_score_0,DIRECT_SCORE_WEIGHTED_MEANS_pre_training_score_1,DIRECT_SCORE_WEIGHTED_MEANS_scores_1,DIRECT_SCORE_WEIGHTED_MEANS_best_score_1,DIRECT_SCORE_WEIGHTED_MEANS_pre_training_score_2,DIRECT_SCORE_WEIGHTED_MEANS_scores_2,DIRECT_SCORE_WEIGHTED_MEANS_best_score_2,DIRECT_SCORE_WEIGHTED_MEANS_pre_training_score_3,DIRECT_SCORE_WEIGHTED_MEANS_scores_3,DIRECT_SCORE_WEIGHTED_MEANS_best_score_3
0,0.503333,"[0.64, 0.7716666666666666, 0.6383333333333333,...",0.771667,0.556667,"[0.8166666666666667, 0.545, 0.72, 0.7983333333...",0.816667,0.523333,"[0.805, 0.7933333333333333, 0.5416666666666666...",0.816667,0.631667,...,0.815000,0.506667,"[0.6366666666666667, 0.6283333333333333, 0.573...",0.675000,0.491667,"[0.6116666666666667, 0.6183333333333333, 0.626...",0.626667,0.678333,"[0.7766666666666666, 0.7733333333333333, 0.773...",0.778333
1,0.531667,"[0.7183333333333334, 0.7366666666666667, 0.808...",0.813333,0.621667,"[0.7583333333333333, 0.7766666666666666, 0.78,...",0.800000,0.616667,"[0.8116666666666666, 0.795, 0.8233333333333334...",0.836667,0.650000,...,0.780000,0.415000,"[0.8283333333333334, 0.79, 0.8166666666666667,...",0.828333,0.505000,"[0.49166666666666664, 0.6783333333333333, 0.54...",0.766667,0.665000,"[0.8066666666666666, 0.825, 0.8133333333333334...",0.825000
2,0.516667,"[0.48333333333333334, 0.48333333333333334, 0.7...",0.776667,0.506667,"[0.7933333333333333, 0.8166666666666667, 0.496...",0.816667,0.658333,"[0.7933333333333333, 0.785, 0.7983333333333333...",0.818333,0.526667,...,0.766667,0.401667,"[0.7716666666666666, 0.7716666666666666, 0.773...",0.773333,0.488333,"[0.7666666666666667, 0.7616666666666667, 0.788...",0.788333,0.381667,"[0.7933333333333333, 0.79, 0.78, 0.77833333333...",0.793333
3,0.478333,"[0.79, 0.6416666666666667, 0.5216666666666666,...",0.790000,0.435000,"[0.7833333333333333, 0.8116666666666666, 0.733...",0.811667,0.666667,"[0.8083333333333333, 0.7966666666666666, 0.811...",0.813333,0.653333,...,0.795000,0.500000,"[0.7766666666666666, 0.79]",0.790000,0.628333,"[0.855, 0.85, 0.8633333333333333, 0.8633333333...",0.863333,0.665000,"[0.8133333333333334, 0.8066666666666666, 0.838...",0.841667
4,0.678333,"[0.49666666666666665, 0.625, 0.783333333333333...",0.783333,0.490000,"[0.8133333333333334, 0.8083333333333333]",0.813333,0.505000,"[0.7783333333333333, 0.5066666666666667, 0.495...",0.778333,0.506667,...,0.846667,0.481667,"[0.785, 0.775, 0.785, 0.7633333333333333, 0.80...",0.821667,0.560000,"[0.755, 0.7616666666666667, 0.7833333333333333...",0.783333,0.446667,"[0.7666666666666667, 0.7883333333333333, 0.773...",0.788333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
len("_scores_0")

9

In [320]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    print(aggregation_method)
    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-21]]
    sub_filtered_rel_cols=[c for c in rel_col if "pre_training" in c]
    result_dict=dict()
    result_dict["by_node_results_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-13]]
    sub_filtered_rel_cols=[c for c in rel_col if "best_score" in c]
    result_dict["by_node_results_best_score"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_best_score"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-9]]
    sub_filtered_rel_cols=[c for c in rel_col if "scores" in c]
    by_node_result_average_dict=dict()
    by_node_result_std_dict=dict()
    by_node_retraining_count_dict=dict()
    by_node_training_count_dict=dict()
    for col in sub_filtered_rel_cols:
        by_node_result_average_dict[col.split("_")[-1]]=np.mean(np.concatenate(training_scores_df[col].dropna().values))
        by_node_result_std_dict[col.split("_")[-1]]=np.std(np.concatenate(training_scores_df[col].dropna().values))
        by_node_training_count_dict[col.split("_")[-1]]=len(np.concatenate(training_scores_df[col].dropna().values))
        by_node_retraining_count_dict[col.split("_")[-1]]= len(training_scores_df[col].dropna().values)
    result_dict["by_node_results_average_scores"]=by_node_result_average_dict
    result_dict["total_average_average_scores"]=np.mean([val for val in by_node_result_average_dict.values()])
    result_dict["by_node_results_std_scores"]=by_node_result_std_dict
    result_dict["total_average_std_scores"]=np.mean([val for val in by_node_result_std_dict.values()])
    result_dict["by_node_results_retraining_count"]=by_node_retraining_count_dict
    result_dict["total_retraining_count"]=np.sum([val for val in by_node_retraining_count_dict.values()])
    result_dict["by_node_results_training_count"]=by_node_training_count_dict
    result_dict["total_training_count"]=np.sum([val for val in by_node_training_count_dict.values()])
    average_score_dict[aggregation_method]=result_dict

DIRECT_MEANS
GRADUAL_GENETIC_ALGORITHM_MATING
DIRECT_GENETIC_ALGORITHM_MATING
GRADUAL_SCORE_WEIGHTED_MEANS
NO_META_LEARNING
GRADUAL_MEANS
DIRECT_SCORE_WEIGHTED_MEANS


In [321]:
total_average_pre_training = []
total_average_best_score = []
total_average_std_scores = []
total_retraining_count = []
total_training_count = []
total_average_average_scores = []
aggregation_names=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    total_average_pre_training.append(val["total_average_pre_training"])
    total_average_best_score.append(val["total_average_best_score"])
    total_average_std_scores.append(val["total_average_std_scores"])
    total_retraining_count.append(val["total_retraining_count"])
    total_training_count.append(val["total_training_count"])
    total_average_average_scores.append(val["total_average_average_scores"])
print(list(zip(aggregation_names,total_training_count)))
sorted_tuples = sorted(
    zip(
        aggregation_names,
        total_average_pre_training,
        total_average_best_score,
        total_average_std_scores,
        total_retraining_count,
        total_training_count,
        total_average_average_scores,
    ),
    key=lambda x: x[2],
)
aggregation_names = [val[0] for val in sorted_tuples]
total_average_pre_training = [val[1] for val in sorted_tuples]
total_average_best_score = [val[2] for val in sorted_tuples]
total_average_std_scores = [val[3] for val in sorted_tuples]
total_retraining_count = [val[4] for val in sorted_tuples]
total_training_count = [val[5] for val in sorted_tuples]
total_average_average_scores = [val[6] for val in sorted_tuples]

[('DIRECT_MEANS', 1012), ('GRADUAL_GENETIC_ALGORITHM_MATING', 3672), ('DIRECT_GENETIC_ALGORITHM_MATING', 589), ('GRADUAL_SCORE_WEIGHTED_MEANS', 872), ('NO_META_LEARNING', 1036), ('GRADUAL_MEANS', 1057), ('DIRECT_SCORE_WEIGHTED_MEANS', 624)]


In [322]:
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Gradual sampled',
 'No MetaL',
 'Gradual weighted',
 'Direct weighted',
 'Direct sampled',
 'Gradual regular',
 'Direct regular']

In [323]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_pre_training,name="Pre-training"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_best_score,name="Best"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_average_scores,name="Average"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_std_scores,name="Training std"))
fig.update_layout(template="none", title=f"Training results by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Training_results_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [324]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_training_count,name="Trainings"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_retraining_count,name="Re-trainings"))
fig.update_layout(template="none", title=f"Training cost by {' '.join(plot_file_term.split('_'))} methods for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Number of executions")
fig.write_image(f"./../figs/Training_cost_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()